In [3]:
import pandas as pd

# Load the CSV
df = pd.read_csv('donnees_vol_multiple.csv')  # use your actual filename if different
df['is_attack'] = 1
# List of columns to keep
columns_to_keep = [
    'mission_id', 'time', 'real_x', 'real_y', 'real_z',
    'vx', 'vy', 'vz', 'ax', 'ay', 'az',
    'roll', 'pitch', 'yaw',
    'wind_x', 'wind_y', 'wind_z',
    'erreur_m', 'velocity',
    'rain', 'fog','snow' ,'is_attack'      

]

# Filter the DataFrame
filtered_df = df[columns_to_keep]

# Save the result
filtered_df.to_csv('donnees_nettoyee_bon.csv', index=False)

print(" Saved cleaned CSV with selected columns to 'donnees_nettoyee.csv'")

 Saved cleaned CSV with selected columns to 'donnees_nettoyee.csv'


In [4]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('donnees_nettoyee_bon.csv')

# Final list of valid flight rows
output_rows = []

# Track current block of rows
current_block = []
current_id = None

# Counter for assigning new clean mission_ids
new_flight_id = 1

# Go through rows one by one
for _, row in df.iterrows():
    row_id = row['mission_id']

    if current_id is None:
        current_id = row_id
        current_block.append(row)
    elif row_id == current_id:
        current_block.append(row)
    else:
        # ID changed, time to process the current block
        if len(current_block) == 21:
            for r in current_block:
                r['mission_id'] = new_flight_id
                output_rows.append(r)
            new_flight_id += 1

        # Start a new block
        current_block = [row]
        current_id = row_id

# Handle the last block
if len(current_block) == 21:
    for r in current_block:
        r['mission_id'] = new_flight_id
        output_rows.append(r)

# Convert to DataFrame
output_df = pd.DataFrame(output_rows)

# Save to CSV
output_df.to_csv('donnees_vol_multiple_filtered_bon.csv', index=False)

print("✅ Saved cleaned flight data with new mission IDs to 'cleaned_flight_data.csv'")

✅ Saved cleaned flight data with new mission IDs to 'cleaned_flight_data.csv'


In [5]:
import pandas as pd

# Charger les données
df = pd.read_csv("donnees_vol_multiple_filtered_bon.csv")

# Liste des vols valides
vols_valides = []

# Vérification des conditions sur chaque vol
for mission_id, groupe in df.groupby("mission_id"):
    groupe_sorted = groupe.sort_values("time")
    premiers_wp = groupe_sorted.head(7)

    # Vérifier les deux conditions
    condition_erreur_7points = (premiers_wp["erreur_m"] > 8).any()
    moyenne_vol = groupe_sorted["erreur_m"].mean()
    condition_moyenne = moyenne_vol >= 18

    # Appliquer le OU : on supprime le vol si l'une des deux conditions est vraie
    if condition_erreur_7points or condition_moyenne:
        continue

    # Sinon, garder ce vol
    vols_valides.append(mission_id)

# Filtrer le DataFrame avec les vols valides
df_final = df[df["mission_id"].isin(vols_valides)]

# Sauvegarder dans un nouveau fichier CSV
df_final.to_csv("donnees_vols_valides.csv", index=False)

# Affichage de la moyenne d'erreur et du nombre de vols gardés
moyenne_finale = df_final["erreur_m"].mean()
print(f"Moyenne finale des erreurs : {moyenne_finale:.2f} mètres")
print(f"Nombre de vols restants : {len(vols_valides)}")
print("Fichier 'donnees_vols_valides.csv' enregistré avec succès.")


Moyenne finale des erreurs : 3.64 mètres
Nombre de vols restants : 128
Fichier 'donnees_vols_valides.csv' enregistré avec succès.


In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import pandas as pd
import numpy as np

def main():
    # 1. Charger les données
    df = pd.read_csv('donnees_vols_valides.csv')

    # 2. Calcul de la matrice de corrélation
    corr = df.corr()
    print("Matrice de corrélation :")
    print(corr)
    print("\n" + "="*80 + "\n")

    # 3. Extraction des paires de variables (sans doublons ni diagonale)
    mask = np.triu(np.ones(corr.shape), k=1).astype(bool)
    corr_pairs = (
        corr.where(mask)
            .stack()
            .reset_index()
            .rename(columns={'level_0': 'Variable1', 'level_1': 'Variable2', 0: 'Corrélation'})
    )

    # 4. Filtrage des paires avec corrélation >= 0.7 en valeur absolue
    high_corr = corr_pairs[corr_pairs['Corrélation'].abs() >= 0.7] \
        .sort_values(by='Corrélation', ascending=False)

    # 5. Affichage des paires à forte corrélation
    print("Paires de variables avec |corrélation| ≥ 0.7 :")
    for _, row in high_corr.iterrows():
        v1, v2, r = row['Variable1'], row['Variable2'], row['Corrélation']
        print(f"  - {v1} ↔ {v2} : corrélation = {r:.3f}")

if __name__ == "__main__":
    main()


Matrice de corrélation :
            mission_id      time    real_x    real_y    real_z        vx  \
mission_id    1.000000  0.949304  0.068975  0.032472  0.115995  0.082677   
time          0.949304  1.000000 -0.008194  0.042917  0.154934  0.049011   
real_x        0.068975 -0.008194  1.000000 -0.259433 -0.329191  0.204356   
real_y        0.032472  0.042917 -0.259433  1.000000  0.341926 -0.076905   
real_z        0.115995  0.154934 -0.329191  0.341926  1.000000 -0.059381   
vx            0.082677  0.049011  0.204356 -0.076905 -0.059381  1.000000   
vy            0.046056  0.050764 -0.032706  0.566704  0.178907  0.006043   
vz           -0.033870 -0.037276  0.169648  0.124251  0.439004  0.014918   
ax            0.026514  0.022191  0.027245 -0.000590 -0.051776  0.060930   
ay            0.034234  0.026704  0.012423 -0.013278  0.007322  0.010105   
az            0.033955  0.048371 -0.053650 -0.005462  0.134811 -0.027248   
roll          0.037604  0.060326 -0.145209 -0.011580  0.091118 

In [7]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import pandas as pd

def main():
    # 1. Charger les données
    df = pd.read_csv('donnees_vols_valides.csv')

    # 2. Calcul de la variance
    variances = df.var()

    # 3. Filtrer les variables avec une variance > 0.01
    filtered_vars = variances[variances > 0.01].sort_values(ascending=False)

    # 4. Affichage
    print("Variables avec une variance > 0.01 :\n")
    for var, var_value in filtered_vars.items():
        print(f"  - {var} : variance = {var_value:.4f}")

if __name__ == "__main__":
    main()


Variables avec une variance > 0.01 :

  - time : variance = 40039019617.8845
  - mission_id : variance = 10083.9780
  - real_x : variance = 84.1219
  - erreur_m : variance = 39.6225
  - velocity : variance = 17.3420
  - real_y : variance = 9.9273
  - yaw : variance = 5.3806
  - real_z : variance = 4.2254
  - az : variance = 1.5651
  - vx : variance = 0.6339
  - vy : variance = 0.3080
  - ay : variance = 0.1969
  - vz : variance = 0.1092
  - ax : variance = 0.0991
  - wind_y : variance = 0.0694
  - wind_x : variance = 0.0681
  - fog : variance = 0.0183
  - snow : variance = 0.0163


In [8]:
import pandas as pd

# Charger le fichier
df = pd.read_csv("donnees_vol_multiple_filtered_bon.csv")

# Réinitialiser l'index
df = df.reset_index(drop=True)

# Garder la valeur is_attack originale par mission_id
is_attack_values = df.groupby('mission_id')['is_attack'].first().reset_index()

# Ajouter un compteur par mission_id (position du waypoint)
df['ligne_id'] = df.groupby('mission_id').cumcount()

# Passer de long à wide : chaque ligne devient un vol à plat
df_flat = df.pivot(index='mission_id', columns='ligne_id')

# Aplatir les colonnes multi-niveaux
df_flat.columns = [f"{col}_{i}" for col, i in df_flat.columns]

# Réinitialiser l'index pour fusionner
df_flat = df_flat.reset_index()

# Fusionner avec les valeurs originales de is_attack
df_flat = pd.merge(df_flat, is_attack_values, on='mission_id', how='left')

# Supprimer toutes les colonnes is_attack_X qui ont été créées par le pivot
cols_to_drop = [col for col in df_flat.columns if col.startswith('is_attack_')]
df_flat = df_flat.drop(columns=cols_to_drop)

# Sauvegarder le fichier aplati
df_flat.to_csv("vols_aplatis_BON.csv", index=False)

print("✅ Vols aplatis avec succès ! (avec un seul is_attack par vol)")

✅ Vols aplatis avec succès ! (avec un seul is_attack par vol)
